In [0]:
#Spark Session
from pyspark.sql import SparkSession
spark = (
        SparkSession
        .builder
        .appName("Joins and Data partitions")
        .getOrCreate()
)

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# Emp Data & Schema
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

dept_data = [
    ["101", "Sales", "NYC", "US", "1000000"],
    ["102", "Marketing", "LA", "US", "900000"],
    ["103", "Finance", "London", "UK", "1200000"],
    ["104", "Engineering", "Beijing", "China", "1500000"],
    ["105", "Human Resources", "Tokyo", "Japan", "800000"],
    ["106", "Research and Development", "Perth", "Australia", "1100000"],
    ["107", "Customer Service", "Sydney", "Australia", "950000"]
]

dept_schema = "department_id string, department_name string, city string, country string, budget string"

In [0]:
# Create emp & dept DataFrame
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)
dept = spark.createDataFrame(data=dept_data, schema=dept_schema)

In [0]:
# Show emp dataframe (ACTION)
emp.show()
dept.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [0]:
# Print Schema
emp.printSchema()
dept.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)

root
 |-- department_id: string (nullable = true)
 |-- department_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- budget: string (nullable = true)



In [0]:
# Get number of partitions for emp
emp.rdd.getNumPartitions()

Out[8]: 8

In [0]:
# Get number of partitions for dept
dept.rdd.getNumPartitions()

Out[10]: 8

In [0]:
# Repartition of data using repartition & coalesce
emp_partitioned = emp.repartition(4)
emp_partitioned.rdd.getNumPartitions()

Out[12]: 4

In [0]:
# Repartition of data using repartition & coalesce
emp_partitioned = emp.coalesce(2)
emp_partitioned.rdd.getNumPartitions()

Out[15]: 2

- The below code implies that the same department_id will lie in the same partition. 
- There is one big de-merit of repartitioning.
- Repartition involves shuffling. 
- In order to make the desired partitions, the data would be moved across the executors

In [0]:
# Repartition based on a specific column
emp_partitioned = emp.repartition(4, "department_id")
emp_partitioned.rdd.getNumPartitions()

Out[16]: 4

**Lets see how the data is moved across partitions**

In [0]:
# Find the partition info for partitions and repartition
from pyspark.sql.functions import spark_partition_id
emp1 = emp.withColumn("partition_num", spark_partition_id())
emp1.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_num|
+-----------+-------------+-------------+---+------+------+----------+-------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|            0|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|            0|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|            1|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|            1|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|            2|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|            2|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|            3|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|            3|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-0

- If we see the above result, the same department_id is belongs to more than one partition number. 
- Eg: department_id '102' belongs to partitions 1,3 and 7 i.e. it is moved across partitions. 
- Now, lets repartition the data to bring all department_id's into its respective same partitions in the below code.

In [0]:
emp1 = emp.repartition(4, "department_id").withColumn("partition_num", spark_partition_id())
emp1.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_num|
+-----------+-------------+-------------+---+------+------+----------+-------------+
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|            0|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|            0|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|            0|
|        014|          107|    Emily Lee| 26|Female| 46000|2019-01-01|            0|
|        016|          107|  Kelly Zhang| 30|Female| 49000|2018-04-01|            0|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|            0|
|        012|          105|   Susan Chen| 31|Female| 54000|2017-02-15|            1|
|        017|          105|  George Wang| 34|  Male| 57000|2016-03-15|            1|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-0

In [0]:
# INNER JOIN datasets
# select e.emp_name, d.department_name, d.department_id, e.salary 
# from emp e inner join dept d on emp.department_id = dept.department_id

df_joined = emp.join(dept, how="inner", on=emp.department_id ==  dept.department_id)
df_joined.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+--------------------+-------+---------+-------+
|employee_id|department_id|         name|age|gender|salary| hire_date|department_id|     department_name|   city|  country| budget|
+-----------+-------------+-------------+---+------+------+----------+-------------+--------------------+-------+---------+-------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|          101|               Sales|    NYC|       US|1000000|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|          101|               Sales|    NYC|       US|1000000|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|          101|               Sales|    NYC|       US|1000000|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|          102|           Marketing|     LA|       US| 900000|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|      

In [0]:
# select only particular columns you need
df_joined.select(emp.name, dept.department_id, dept.department_name, emp.salary).show() 

+-------------+-------------+--------------------+------+
|         name|department_id|     department_name|salary|
+-------------+-------------+--------------------+------+
|     John Doe|          101|               Sales| 50000|
|   Jane Smith|          101|               Sales| 45000|
|James Johnson|          101|               Sales| 70000|
|    Bob Brown|          102|           Marketing| 55000|
|    Alice Lee|          102|           Marketing| 48000|
|     Kate Kim|          102|           Marketing| 51000|
|    Grace Kim|          102|           Marketing| 53000|
|    Jack Chan|          103|             Finance| 60000|
|    Jill Wong|          103|             Finance| 52000|
|      Tom Tan|          103|             Finance| 58000|
|  Steven Chen|          103|             Finance| 62000|
|     Lisa Lee|          104|         Engineering| 47000|
|   David Park|          104|         Engineering| 65000|
|    Nancy Liu|          104|         Engineering| 50000|
|   Susan Chen

In [0]:
# Shorten the table names 'emp' and 'dept' with alias
df_joined = emp.alias("e").join(dept.alias("d"), how="inner", on=emp.department_id == dept.department_id)
df_joined.select("e.name", "d.department_id", "d.department_name", "e.salary").show() 

+-------------+-------------+--------------------+------+
|         name|department_id|     department_name|salary|
+-------------+-------------+--------------------+------+
|     John Doe|          101|               Sales| 50000|
|   Jane Smith|          101|               Sales| 45000|
|James Johnson|          101|               Sales| 70000|
|    Bob Brown|          102|           Marketing| 55000|
|    Alice Lee|          102|           Marketing| 48000|
|     Kate Kim|          102|           Marketing| 51000|
|    Grace Kim|          102|           Marketing| 53000|
|    Jack Chan|          103|             Finance| 60000|
|    Jill Wong|          103|             Finance| 52000|
|      Tom Tan|          103|             Finance| 58000|
|  Steven Chen|          103|             Finance| 62000|
|     Lisa Lee|          104|         Engineering| 47000|
|   David Park|          104|         Engineering| 65000|
|    Nancy Liu|          104|         Engineering| 50000|
|   Susan Chen

In [0]:
# LEFT OUTER JOIN datasets
# select e.emp_name, d.department_name, d.department_id, e.salary 
# from emp e left outer join dept d on emp.department_id = dept.department_id

df_joined = emp.alias("e").join(dept.alias("d"), how="left_outer", on=emp.department_id == dept.department_id)
df_joined.select("e.name", "d.department_id", "d.department_name", "e.salary").show() 

+-------------+-------------+--------------------+------+
|         name|department_id|     department_name|salary|
+-------------+-------------+--------------------+------+
|     John Doe|          101|               Sales| 50000|
|   Jane Smith|          101|               Sales| 45000|
|    Bob Brown|          102|           Marketing| 55000|
|    Alice Lee|          102|           Marketing| 48000|
|    Jack Chan|          103|             Finance| 60000|
|    Jill Wong|          103|             Finance| 52000|
|James Johnson|          101|               Sales| 70000|
|     Lisa Lee|          104|         Engineering| 47000|
|     Kate Kim|          102|           Marketing| 51000|
|      Tom Tan|          103|             Finance| 58000|
|   David Park|          104|         Engineering| 65000|
|   Susan Chen|          105|     Human Resources| 54000|
|    Emily Lee|          107|    Customer Service| 46000|
|    Brian Kim|          106|Research and Deve...| 75000|
|  Kelly Zhang

In [0]:
# WRITE the final dataset
df_joined.select("e.name", "d.department_name", "d.department_id","e.salary").write.format("csv").save("data/output/7/emp_joined.csv")

- Bonus TIP
- Joins with cascading conditions
- Join with Department_id and only for department_id's '101' or '102'
- Join with not null/null conditions

In [0]:
# CASCADING
df_final = emp.join(dept, how = "left_outer",
                          on = (emp.department_id == dept.department_id) & ((emp.department_id == '101') | (emp.department_id == '102'))
                           )
df_final.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+---------------+----+-------+-------+
|employee_id|department_id|         name|age|gender|salary| hire_date|department_id|department_name|city|country| budget|
+-----------+-------------+-------------+---+------+------+----------+-------------+---------------+----+-------+-------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|          101|          Sales| NYC|     US|1000000|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|          101|          Sales| NYC|     US|1000000|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|          102|      Marketing|  LA|     US| 900000|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|          102|      Marketing|  LA|     US| 900000|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|         null|           null|null|   null|   null|
|        006|          1

- In the above result we see that, the joining condition is based on only department_id's '101' or '102'. 
- For all other department_id's, the values for the join condition is 'null'



In [0]:
# Eliminate null values
df_final = emp.join(dept, how = "left_outer",
                          on = (emp.department_id == dept.department_id)
                          & (emp.department_id.isNotNull())
                           )
df_final.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+--------------------+-------+---------+-------+
|employee_id|department_id|         name|age|gender|salary| hire_date|department_id|     department_name|   city|  country| budget|
+-----------+-------------+-------------+---+------+------+----------+-------------+--------------------+-------+---------+-------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|          101|               Sales|    NYC|       US|1000000|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|          101|               Sales|    NYC|       US|1000000|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|          102|           Marketing|     LA|       US| 900000|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|          102|           Marketing|     LA|       US| 900000|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|      